# This does not work in the VSCode environment as-is.  The animation is not visible at all.

In [1]:
%matplotlib widget
from electrical_analysis.output.plot_tools import *
from electrical_analysis.waveform.tracemath import dB

import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import solve as linsolve


In [3]:
c = 3.e10/np.sqrt(4) # cm/s
Zs = 50
Zo = 50
Zl = 25

K = 101 # Spatial Points
N = 2001

l = 5. # 5 cm (line length)

t, dt = np.linspace(0., 1.5e-9, N, retstep=True)
# dt = 1 ps
x, dz = np.linspace(0., l, K, retstep=True)

# Initialize the matrices.
Av = np.zeros((K+2,K+2)) # x vv next
Bv = np.zeros((K+2,K+2)) # x vv previous
Cv = -np.eye(K+2,K-1) + np.eye(K+2,K-1,-1)  # x iv previous
Dv = np.zeros(K+2) # vs

# source voltage
def unitstep(t, t0):
    return (t >= t0) * 1.0

def unitramp(t, t0):
    return unitstep(t,t0) * (t-t0)

tstart = 25e-12 
tstop = 500e-12 
tramp = 25e-12 
vs = (unitramp(t, tstart) - unitramp(t,tstart+tramp) - unitramp(t,tstop) + unitramp(t,tstop+tramp))/tramp

# Fill the matrices.  (These are static in this form.)
v_coeff = dz / (c * Zo * dt)
v_coeff_by_8 = v_coeff / 8.
v_coeff_by_8_times_3 = 3. * v_coeff_by_8

Av[0][0] = v_coeff_by_8_times_3
Av[0][1] = v_coeff_by_8
Av[0][K] = -0.5

Bv[0][0] = v_coeff_by_8_times_3
Bv[0][1] = v_coeff_by_8
Bv[0][K] = 0.5

for k in range(1,K-1):
    Av[k][k] = v_coeff
    Bv[k][k] = v_coeff

Av[K-1][K-2] = v_coeff_by_8
Av[K-1][K-1] = v_coeff_by_8_times_3
Av[K-1][K+1] = 0.5

Bv[K-1][K-2] = v_coeff_by_8
Bv[K-1][K-1] = v_coeff_by_8_times_3
Bv[K-1][K+1] = -0.5

# source equation fill
Av[K][0] = 1./Zs 
Av[K][K] = 1.
Dv[K] = 1./Zs

# load equation fill
Av[K+1][K-1] = 1./Zl 
Av[K+1][K+1] = -1.

############################################################
############################################################
############################################################
# Current update equations
i_coeff = c*dt/(Zo*dz)
Ai = np.eye(K-1,K-1) # not used, just here for illustrative purposes
Bi = np.eye(K-1,K-1) # not used
Ci = np.eye(K-1,K+2) - np.eye(K-1,K+2,1) # times v^{n+1/2}

############################################################
############################################################
############################################################

# Time stepping
vv = np.zeros(K+2) # voltage vector + auxiliary load current variables
iv = np.zeros(K-1) # current vector

N = len(t)
nn = 0
vrecord = []
irecord = []

fig = plt.figure()
a = fig.add_subplot(1,1,1)
#p = PlotObject()
#a = p.axes
L, = a.plot(x, vv[:-2]); a.grid(True)
a.set_ylim(-0.5, 1.5)
a.set_xlabel('tline location (cm)')
a.set_ylabel('voltage (V)')

print ("Av condition = ", np.linalg.cond(Av))
print (Av[0:3][:,0:3])
for ti, vsi in zip(t, vs):
    #print "%d of %d" % (nn+1, N)
    nn += 1
    
    # update voltage n+1/2
    y0 = np.dot(Bv, vv)
    y1 = np.dot(Cv, iv)
    y2 = Dv*vsi
    y = y0 + y1 + y2    
    vv = np.linalg.solve(Av, y)
    vrecord.append(vv)
    
    # update current n+1
    iv = iv + i_coeff*np.dot(Ci,vv)
    irecord.append(iv)
    
    # update the plot
    L.set_ydata(vv[:-2])
    #a.set_ylim(np.min(vv[:-2]),np.max(vv[:-2]))
    a.set_title('t = %f ps' % (ti * 1e12))
    #p.refresh()
    fig.canvas.draw()
    #time.sleep(0.1)
#    if nn % 5 == 0:
#        p.figure.savefig('tmp/tline_%04d.png' % nn)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Av condition =  29.072616697588714
[[0.03333333 0.01111111 0.        ]
 [0.         0.08888889 0.        ]
 [0.         0.         0.08888889]]
